In [ ]:
import logging
import psycopg2
import requests
from pyspark.sql import functions as F
from pyspark.sql.types import IntegerType, DecimalType, StringType, DateType

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Securely retrieve credentials using Databricks utilities
postgres_user = dbutils.secrets.get(scope="my_scope", key="postgres_user")
postgres_password = dbutils.secrets.get(scope="my_scope", key="postgres_password")
mysql_user = dbutils.secrets.get(scope="my_scope", key="mysql_user")
mysql_password = dbutils.secrets.get(scope="my_scope", key="mysql_password")
sqlserver_user = dbutils.secrets.get(scope="my_scope", key="sqlserver_user")
sqlserver_password = dbutils.secrets.get(scope="my_scope", key="sqlserver_password")
fraud_api_key = dbutils.secrets.get(scope="my_scope", key="fraud_api_key")
credit_api_key = dbutils.secrets.get(scope="my_scope", key="credit_api_key")

# Load data from Unity Catalog tables
try:
    customer_demo_df = spark.table("catalog.demographics_db.customer_demographics_table")
    policy_data_df = spark.table("catalog.policy_db.policy_table")
    claims_data_df = spark.table("catalog.claims_db.claims_table")
except Exception as e:
    logger.error(f"Error loading data from Unity Catalog: {e}")
    raise

# Fetch Fraud Score from REST API
def fetch_fraud_score(customer_id):
    try:
        response = requests.get(
            "https://fraudscore.api/endpoint",
            headers={"Authorization": f"Bearer {fraud_api_key}"},
            params={"customer_id": customer_id}
        )
        response.raise_for_status()
        return response.json().get("fraud_score", None)
    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching fraud score for customer {customer_id}: {e}")
        return None

# Fetch Credit Score from REST API
def fetch_credit_score(customer_id):
    try:
        response = requests.get(
            "https://creditscore.api/endpoint",
            headers={"Authorization": f"Bearer {credit_api_key}"},
            params={"customer_id": customer_id}
        )
        response.raise_for_status()
        return response.json().get("credit_score", None)
    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching credit score for customer {customer_id}: {e}")
        return None

# Join policy data with customer demographics
policy_demo_df = policy_data_df.join(customer_demo_df, "Customer_ID", "inner")

# Join the result with claims data
policy_claims_df = policy_demo_df.join(claims_data_df, "Policy_ID", "inner")

# Aggregate data at customer level
agg_customer_df = policy_claims_df.groupBy("Customer_ID").agg(
    F.count("Claim_ID").alias("Total_Claims"),
    F.avg("Claim_Amount").alias("Average_Claim_Amount"),
    F.max("Claim_Date").alias("Recent_Claim_Date"),
    F.countDistinct("Policy_ID").alias("Policy_Count")
)

# Calculate derived fields
derived_df = agg_customer_df.withColumn("Age", F.datediff(F.current_date(), F.col("Date_of_Birth")) / 365) \
    .withColumn("Claim_To_Premium_Ratio", F.when(F.col("Total_Premium_Paid") != 0, F.col("Claim_Amount") / F.col("Total_Premium_Paid")).otherwise(0)) \
    .withColumn("Claims_Per_Policy", F.when(F.col("Policy_Count") != 0, F.col("Total_Claims") / F.col("Policy_Count")).otherwise(0)) \
    .withColumn("Retention_Rate", F.lit(0.85)) \
    .withColumn("Cross_Sell_Opportunities", F.lit("Multi-Policy Discount, Home Coverage Add-on")) \
    .withColumn("Upsell_Potential", F.lit("Premium Vehicle Coverage"))

# Fetch fraud and credit scores
fraud_scores = derived_df.select("Customer_ID").rdd.map(lambda row: (row["Customer_ID"], fetch_fraud_score(row["Customer_ID"])))
credit_scores = derived_df.select("Customer_ID").rdd.map(lambda row: (row["Customer_ID"], fetch_credit_score(row["Customer_ID"])))

fraud_scores_df = spark.createDataFrame(fraud_scores, ["Customer_ID", "Fraud_Score"])
credit_scores_df = spark.createDataFrame(credit_scores, ["Customer_ID", "Credit_Score"])

# Join fraud and credit scores with derived data
final_df = derived_df.join(fraud_scores_df, "Customer_ID", "left").join(credit_scores_df, "Customer_ID", "left")

# Add AI-driven insights
final_df = final_df.withColumn("Churn_Probability", F.lit(0.25)) \
    .withColumn("Next_Best_Offer", F.lit("Additional Life Coverage")) \
    .withColumn("Claims_Fraud_Probability", F.lit(0.10)) \
    .withColumn("Revenue_Potential", F.lit(12000.00))

# Write the final data to Unity Catalog target table
try:
    spark.sql("DROP TABLE IF EXISTS catalog.customer_dw.customer_360")
    final_df.write.format("delta").mode("overwrite").saveAsTable("catalog.customer_dw.customer_360")
    logger.info("Data successfully written to Unity Catalog target table.")
except Exception as e:
    logger.error(f"Error writing data to Unity Catalog target table: {e}")
    raise
